In [ ]:
from MovieRecommend import Movies

In [ ]:
mov = Movies()

In [ ]:
mov.print_files()

In [ ]:
mov.preview_files()

In [ ]:
new_df = mov.get_df()

In [ ]:
mov.train_model()

In [ ]:
mov.preview_files()

In [ ]:
df = mov.get_df()

In [ ]:
mov.print_files()

In [ ]:
print(df.columns)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import pandas as pd

numerical_cols = ['BoxOffice', 'Metascore', 'Runtime', 'Year', 'imdbRating', 'Days to View', 'Days Since Release']
categorical_cols = ['Rated', 'Format', 'Is_cinema', 'Is_weekday']
plots = ['Plot']

mlb_cols = ['Genre', 'Bag_of_words']
#mlb_cols = 'Genre'

form_cols = ['Genre', 'Actors', 'Director', 'Production', 'Writer']

#for col in form_cols:
#    col_str = col + "_"
#    in_cols = [col for col in df.columns if col_str in col]
#    for col2 in in_cols:
#        categorical_cols.append(col2)

print(categorical_cols)
features = numerical_cols + categorical_cols + plots + mlb_cols

for i in features:
    df[i] = df[i].replace('N/A',np.NaN)
    df[i] = df[i].replace('',np.NaN)
    df[i] = df[i].replace(' ',np.NaN)
    df[i] = df[i].replace('missing_value',np.NaN)

numerical_transformer = SimpleImputer(missing_values=np.NaN,
                                      strategy='median')

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values=np.NaN,
                              strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

#mlb_transformer = MultiLabelBinarizer()

#tfidf_transformer = TfidfVectorizer()

#tfidf_transformer = TfidfVectorizer()
#tfidf_transformer.fit(df['Plot'])
#print(tfidf_transformer)
#j = tfidf_transformer.transform(df['Plot'])

#j_df = pd.DataFrame(j.toarray(), columns = tfidf_transformer.get_feature_names_out())
#res = pd.concat([df, j_df], axis = 1)
#print(res.head())

#features += tfidf_transformer.get_feature_names_out().tolist()
#print(features)

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
class MultiHotEncoder(BaseEstimator, TransformerMixin):
    """Allow MultiLabelBinarizer to work with multiple input columns."""
    def __init__(self):
        self.mlbs = list()
        self.n_columns = 0
        self.categories_ = self.classes_ = list()

    def fit(self, X:pd.DataFrame, y=None):
        for i in range(X.shape[1]): # X can be of multiple columns
            mlb = MultiLabelBinarizer()
            mlb.fit(X.iloc[:,i])
            self.mlbs.append(mlb)
            self.classes_.append(mlb.classes_)
            self.n_columns += 1
        return self
    
    def transform(self, X:pd.DataFrame):
        if self.n_columns == 0:
            raise ValueError('Please fit the transformer first.')
        if self.n_columns != X.shape[1]:
            raise ValueError(f'The fit transformer deals with {self.n_columns} columns '
                              f'while the input has {X.shape[1]}'
                              )
        result = list()
        for i in range(self.n_columns):
            result.append(self.mlbs[i].transform(X.iloc[:,i]))
        result = np.concatenate(result, axis=1)
        return result

In [ ]:
mlb_transformer = MultiHotEncoder()

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols),
        #('tfidf', Tfidf, plots)
        ('mlb', mlb_transformer, mlb_cols)
    ]
)

#x = res[features]
#y = res['True Rating']

x=df[features]
y=df['True Rating']

X_train, X_val, y_train, y_val = train_test_split(x, y, random_state = 0)
#print(X_train)
#print(x[tfidf_transformer.get_feature_names()])
print(type(df['Plot']))
#print(j)

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class Tfidf(BaseEstimator, TransformerMixin):
    def __init__(self, variables=None) -> None:
        if not isinstance(variables, list):
            self.variables = [variables]
        else:
            self.variables = variables
       
    def fit(self, X, y=None):
        self.tfidf_transformer = TfidfVectorizer()
        X = X[self.variables].squeeze()
        #print(type(X[self.variables]))
        self.tfidf_transformer.fit(X)
        self.feature_names = self.tfidf_transformer.get_feature_names_out()
        return self
    
    def transform(self, X):
        X = X[self.variables].squeeze()
        #print(X)
        j = self.tfidf_transformer.transform(X)
        print(j)
        j_df = pd.DataFrame(j.toarray(),
                            columns = self.feature_names)
        print(j_df)
        return j_df

In [ ]:
X_train_xfm = preprocessor.fit_transform(X_train)

In [ ]:
test = Tfidf(variables=['Plot'])
test.fit(X_train)
j = test.transform(X_train)

In [ ]:
from xgboost import XGBRegressor

params = {"n_estimators": [50, 100, 150, 200, 300, 500, 750, 1000],
          #"max_features": ['auto', 'sqrt', 'log2'],
          "grow_policy": ['depthwise', 'lossguide'],
          "max_depth": [3, 4, 5],
          "min_child_weight": [1, 2, 5, 10],
          "gamma": [0.5, 1, 1.5, 2, 5],
          "subsample": [0.4, 0.6, 0.8, 1.0],
          "colsample_bytree": [0.6, 0.8, 1.0]
          }

xgb_model = XGBRegressor(n_estimators=1000,
                         learning_rate=0.05,
                         n_jobs=-2)

grid = GridSearchCV(xgb_model, 
                    param_grid=params,
                    scoring="r2",
                    verbose=2,
                    n_jobs = -3)

#xgb_pipeline = Pipeline(steps=[
#    ('preprocessor', preprocessor),
#    ('model', xgb_model)
#])

grid_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('tfidf', Tfidf),
    ('model', grid)
])

In [ ]:
new_search = False

if new_search:
    grid_pipeline.fit(X_train, y_train)
    good_boys = grid.best_params_
else:
    #no nlp:
    #good_boys = {'colsample_bytree': 0.6, 'gamma': 1.5, 'max_depth': 3, 'max_features': 'auto', 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 1.0}
    # nlp: 
    #good_boys = {'colsample_bytree': 0.6, 'gamma': 5, 'max_depth': 3, 'max_features': 'auto', 'min_child_weight': 1, 'n_estimators': 500, 'subsample': 0.8}
    good_boys = {'colsample_bytree': 0.8, 'gamma': 1.5, 'grow_policy': 'depthwise', 'max_depth': 3, 'min_child_weight': 2, 'n_estimators': 100, 'subsample': 0.8}

In [ ]:
good_boys = {'colsample_bytree': 0.8, 
             'gamma': 1.5, 
             'grow_policy': 'depthwise', 
             'max_depth': 3, 
             'min_child_weight': 2, 
             'n_estimators': 100, 
             'subsample': 0.8}

In [ ]:
print(good_boys)

In [ ]:
#good_boys = {'colsample_bytree': 0.6, 'gamma': 5, 'max_depth': 3, 'max_features': 'auto', 'min_child_weight': 1, 'n_estimators': 500, 'subsample': 0.8}

xgb_model = XGBRegressor(n_estimators=good_boys['n_estimators'],
                         learning_rate=0.05,
                         colsample_bytree=good_boys['colsample_bytree'],
                         gamma=good_boys['gamma'],
                         max_depth=good_boys['max_depth'],
                         #max_features=good_boys['max_features'],
                         min_child_weight=good_boys['min_child_weight'],
                         subsample=good_boys['subsample'],
                         grow_policy=good_boys['grow_policy'],
                         n_jobs=-2)
xgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('tfidf', Tfidf),
    ('model', xgb_model)
])

In [ ]:
preprocessor.fit(X_train, y_train)

X_train_xfm = preprocessor.fit_transform(X_train)
X_val_xfm = preprocessor.transform(X_val)

xgb_pipeline.fit(X_train, y_train,
                model__early_stopping_rounds=50,
                model__eval_set=[(X_val_xfm, y_val)])

preds = xgb_pipeline.predict(X_val)

score = mean_absolute_error(preds, y_val)
print("MAE: " + str(score))

score2 = mean_squared_error(preds, y_val)
print("MSE: " + str(score2))

In [ ]:
print(new_df['True Rating'])

In [ ]:
print(X_train_xfm)

In [ ]:
import seaborn as sns

sns.histplot(new_df['True Rating'], bins=3)